# Converter for NASA EarthData  #
#### *For the conversion of Astronomers Julian Day dates to Gregorian Calendar Day and Times* ##

AJD in the format 2459001.966667

Equating to GD 2020/06/01 11:12:00



In [ ]:
import datetime
import pandas as pd
import openpyxl

In [ ]:
def AJD_converter(AJD):
    
    AJD = "{0:.6f}".format(AJD)    # Formats any Astronomers Julian Day value input into the function to have 6 points after the decimal 
                                   # e.g. 2459001.966667 runs through the function the same way that 2459022.900000 would
                                   # Without this line a value such as 2459022.935000 would be treated as 2459022.935 and the calculation of the total_seconds would return an error
    
    AJD_split = str(AJD).split('.')    # Treats AJD as a string and splits AJD at the decimal point
                                       # So the numbers to the left of the decimal can be used for the calculation of the date (yyy/mm/dd)
                                       # And the numbers to the right of the decimal can be used for the calculation of the time (hh:mm:ss)
    ############
    ### time ###
    ############

    split_time_nums = [int(i) for i in str(AJD_split[1])]       # Splits time element of number (RHS of decimal) into individual numbers i.e. 917082 into 9 1 7 0 8 2 
    
    total_seconds = round((split_time_nums[0] *8640) + (split_time_nums[1] *864) + (split_time_nums[2] *86.4) + (split_time_nums[3] *8.64) + (split_time_nums[4] *0.864) + (split_time_nums[5] *0.0864))
                                                                
                                                                # 'total_seconds' is the sum of seconds calculated through the conversion of the decimal part of the Astronomers Julian day to seconds 

                                                                # The calculation uses the following conversions: 
                                                                # 0.1 = 2.4 hours or 144 minutes or 8640 seconds
                                                                # 0.01 = 0.24 hours or 14.4 minutes or 864 seconds
                                                                # 0.001 = 0.024 hours or 1.44 minutes or 86.4 seconds
                                                                # 0.0001 = 0.0024 hours or 0.144 minutes or 8.64 seconds
                                                                # 0.00001 = 0.00024 hours or 0.0144 minutes or 0.864 seconds
                                                                # 0.000001 = 0.000024 hours or 0.00144 minutes or 0.0864 seconds

    if int(AJD_split[1]) >= 500000:
        a = datetime.timedelta(seconds=(total_seconds-43200))    # -43200 to take 23:02 (pm) to 11:02 (am)
    elif int(AJD_split[1]) < 500000:
        a = datetime.timedelta(seconds=(total_seconds+43200))    # +43200 to take 11:02 (am) to 23:02 (pm)
                                   
                                                                # Astronomers Julian Day Runs from midday to midday, whereas a Gregorian day runs from 00:00 to 23:59 
                                                                # As such:
                                                                    # AJD: .000000 --> GD: 12:00:00
                                                                    # AJD: .500000 --> GD: 00:00:00
                                                                    # AJD: .999993 --> GD: 11:59:59
                                                               
                                                                # 0.500000 (AJD) is equaivalent to 43200 seconds in Gregorian Date time

                                                                # Adding or subtracting 43200 seconds (12 hours) allows python to count from 00:00 rather than 12:00 midday
                                                                # As such, where 'AJD_split[1]' greater than (.)500000, 43200 seconds needs to be subtracted so the count of seconds starts from midnight
                                                                # Conversely, where 'AJD_split[1]' less than (.)500000, 43200 seconds needs to be added so the count of seconds starts from midday


   

    #a = datetime.timedelta(seconds=(total_seconds))        # datetime.timedelta(seconds = number of seconds) converts a number of seconds to hh:mm:ss
    
    ############
    ### date ### 
    ############
        
    base_date = datetime.date(2010,1,1)                        # 1st January 2010 is used as a base date with a known Astronomers Julian Day Date
    base_date_jd = 2455198                                     # This base AJD date is used with the 'datetime.timedelta(diff)' command to calculate the new AJD date
    
    new_date_jd = int(AJD_split[0])                            # 'new_date_jd' is the Julian Day date portion from AJD_split to converted to a new GD date

    
    diff = new_date_jd - base_date_jd                          # Calculates the numerical difference between the new AJD date and the base AJD date

    if int(AJD_split[1]) >= 500000:                                     # Where total_seconds >= 500000 (prior to adding or subtracting 43200) the next Gregorian day has been reached 
        new_date = (datetime.timedelta(diff+1)) + base_date             # Where this is the case, '1' is added to the date to account for the number of seconds in the day surpassing 12 hours (43200s)
    elif int(AJD_split[1]) < 500000:                                    # 'new_date' is calculated using 'datetime.timedelta(diff)' which outputs a date --> 'diff' number of days after the 'base_date' 
        new_date = (datetime.timedelta(diff)) + base_date
   

    #new_date = (datetime.timedelta(diff+1)) + base_date
    #print(new_date)
    #return (new_date)

    ###########################
    ###     date + time     ###
    ### yyyy/mm/dd hh:mm:ss ###
    ###########################

    date = new_date
    time = str(a)

    output_gregorian_date = (str(date) +' '+ time)
    
    #print (output_gregorian_date)
    return output_gregorian_date

### Assign the path of the variable 'dates_to_convert' 
#### *NB: In this code the file containing the AJD dates to be converted must be an excel (.xlsx) document*

In [ ]:
##### Use the # key to uncomment out the relevant line for your dataset (Excel or csv) #### ???? ####

dates_to_convert = pd.read_excel('C://Users//rfindler//AJD_Converter/statistics_for_outlier_pattern.xlsx',usecols=['Date'])

# Value/Name assigned to 'usecols' must match the column header for the column containing the AJD dates to be converted

In [ ]:
converted_dates = dates_to_convert.Date.apply(AJD_converter)

# Convention:

# a = b.c.apply(d)

# a = name of output converted variable 
# b = name of variable to be used
# c = name of column within variable, where the data to be used for the fucntion is
# d = function to be used

### Check the inputs for a,b,c in: 
### df.insert(a,b,c)

#### Where:

#### a = Column number for new column (Python uses indexing, so A = 0, B = 1, etc.)

#### b = Name for new column

#### c = Variable to be input into new column



#### *NB: DON'T FORGET TO CLOSE THE EXCEL FILE IF YOU HAVE IT OPEN, AS THIS WILL PRODUCE A PERMISSION ERROR*

In [ ]:
df = pd.read_excel('C://Users//rfindler//AJD_Converter/statistics_for_outlier_pattern.xlsx')
df.insert(1,"Gregorian Dates test 5",converted_dates)
#print(df.head)
df.to_excel(r'C://Users//rfindler//AJD_Converter/statistics_for_outlier_pattern.xlsx',index=False)

## For reference:
Information used for the creation of this code was obtained from:
* https://www.aavso.org/sites/default/files/images/jdnumbertable.gif
* https://ssd.jpl.nasa.gov/tools/jdc/#/jd
* https://forums.techguy.org/threads/julian-day-into-date-and-hh-mm-ss.1002353/

